# 單一樣本假設檢定
### 陳威呈、戴瑜萱
---

In [1]:
from scipy import stats
import numpy as np
import pandas as pd
from scipy.stats import norm
import statsmodels.stats.weightstats as stests

# 比例檢定
## 範例
```
某投資顧問公司欲了解民眾的網路成癮程度，了解使用者的差異性，以提供遊戲業者參考使用，蒐集了一份資料，
資料為喜歡玩線上遊戲的男性人數，受訪男性有 350 人而高度網路使用者則紀錄為 1，非高度網路使用者則紀錄為 0。
我們欲檢定男性的非高度網路使用者之比例是否大於 70%。
```

In [2]:
s = pd.read_csv(r"C:\Users\user\Desktop\R 改 python\11_假說檢定\11a_one-sample-test-data\one-sample-test\s.csv")
s = s[['MALE']]
s

,MALE
0,1
1,0
2,0
3,1
4,1
...,...
345,0
346,1
347,0
348,0


+ 將數值變數轉換為因子變數

In [3]:
s['MALE'] = s['MALE'].astype('category')

###  單樣本比例檢定
+ [scipy.stats.binom_test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom_test.html)
+ 顯著水準 : 0.05
+ 虛無假設 : p $\geq$ 0.7
+ 對立假設 : p < 0.7

### 法一

In [4]:
# 創頻率表
table = np.bincount(s['MALE'])                              # 計算數組中每個唯一值的頻數
print(table)                                                # 0:267；1:83

# 單樣本比例檢定
result = stats.binom_test(table, p=0.7, alternative='less') # 指定對立假設為 p < 0.7
print("P值：", result)  

[267  83]
P值： 0.996288755310694


### 法二

In [5]:
count = len(s[s["MALE"] == 0])

# 單樣本比例檢定
result = stats.binom_test(count, n=350, p=0.7, alternative='less') # 指定對立假設為 p < 0.7
print("P值：", result)                                       

P值： 0.996288755310694


+ 此為左尾檢定，Python 並沒有比例檢定單邊信賴區間的函式，所以只能自己寫

1. 依照男性資料所計算非高度網路使用者的比例

In [6]:
phat = len(s[s["MALE"] == 0])/len(s["MALE"])
phat

0.7628571428571429

2. $z_\alpha \sqrt{\frac{\hat p \;(1-\hat p)}{n}}$

In [7]:
# from scipy.stats import norm

percentile = norm.ppf(0.95)
print(percentile)

se = (phat*(1-phat)/len(s["MALE"]))**0.5
print(se)

1.6448536269514722
0.0227348770739697


3. p 的 95% CI (單尾上界) :
$$[\; 0,\;\hat p + z_\alpha \sqrt{\frac{\hat p \;(1-\hat p)}{n}} \;]$$

In [8]:
ci = [0, phat + percentile*se]
print("95% 信賴區間：", ci)

95% 信賴區間： [0, 0.8002526878705578]


```
由以上結果顯示，
依照男性資料所計算非高度網路使用者的比例為 0.7628571(76.28%)，
95% 信賴區間為 (0, 0.800253)，其信賴區間包含 0.7，
且 p 值為 0.9963，因為 p 值大於顯著水準，所以不拒絕虛無假設，
非高度網路使用者比例大於 70%
```

# 平均數或中位數檢定整理
+ n large, variance known or unknown => z-test (誤用 t-test 問題不大)
+ n small, normal & variance known => z-test
+ n small, normal & variance unknown => t-test
+ n small, maybe non-normal but symmetric => Wilcoxon signed rank test (備註1)
+ n small, maybe asymmetric => sign test (備註2)

# 平均數檢定
今天要抽查某咖啡店的咖啡 ( Coffee 為常態分布)

## 雙尾
a. 咖啡店號稱每一杯都是 250 公克 (雙尾檢定) 抽查結果如下

In [9]:
coffee = pd.DataFrame({'gram': [254.1, 253.5, 257.3, 250.6, 243.5, 244.6, 251.5,
                                251.4, 251.7, 242.5, 254.5, 250.9, 251.4, 248.2,
                                249.4, 248.3, 246.9, 249.6, 250.5, 247.5]})
coffee.head()

,gram
0,254.1
1,253.5
2,257.3
3,250.6
4,243.5


+ 顯著水準 : 0.05
+ 虛無假設 : $\mu$ = 250
+ 對立假設 : $\mu \neq$ 250
+ [scipy.stats.ttest_1samp](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_1samp.html)
+ [statsmodels.stats.weightstats.DescrStatsW.tconfint_me](https://www.statsmodels.org/dev/generated/statsmodels.stats.weightstats.DescrStatsW.tconfint_mean.html)

In [10]:
# import statsmodels.stats.weightstats as stests

result = stats.ttest_1samp(coffee['gram'], popmean=250)
ci = stests.DescrStatsW(coffee['gram']).tconfint_mean(alpha=0.05)

print("T 統計量：", result.statistic)
print("P 值：", result.pvalue)
print("95% 信賴區間：", ci)

T 統計量： -0.12620567027026983
P 值： 0.900895074159803
95% 信賴區間： (248.1536556908079, 251.63634430919205)


我們可以發現 p-value 大於 0.05，因此我們不能拒絕 H0
，也就是說我們沒有證據推翻咖啡每杯是 250 克的說法。

## 單尾
b. 咖啡店號稱每一杯都 250 克以上 (左尾檢定)
+ 顯著水準 : 0.05
+ 虛無假設 : $\mu \geq$  250
+ 對立假設 : $\mu$ < 250

In [11]:
result = stats.ttest_1samp(coffee['gram'], popmean=250, alternative='less')
ci = stests.DescrStatsW(coffee['gram']).tconfint_mean(alpha=0.05, alternative='smaller')
print("T 統計量：", result.statistic)
print("P 值：", result.pvalue)
print("95% 信賴區間：", ci)

T 統計量： -0.12620567027026983
P 值： 0.4504475370799015
95% 信賴區間： (-inf, 251.33359578433334)


我們可以發現 p-value 大於 0.05，因此我們不拒絕 H0，也就是說我們沒有證據推翻咖啡每杯都是 250 克以上的說法。

# 中位數檢定

## 範例1
```
泡泡清潔用品公司宣稱其所生產的沐浴乳其重量的中位數為 200 公克，
今隨機抽查 12 瓶，得重量如下：211 203 206 213 192 205 195 198 204 216 207 222，
顯著水準是0.05，檢定該公司的宣稱是否正確？
```

In [12]:
a = pd.read_csv(r"C:\Users\user\Desktop\R 改 python\11_假說檢定\11a_one-sample-test-data\one-sample-test\a.csv")
a

,g
0,216
1,213
2,211
3,207
4,206
5,205
6,204
7,203
8,198
9,195


In [13]:
median = a.median(axis=0)
mean = a.mean(axis=0)
print("中位數：", median, '\n')
print("均值：", mean)

中位數： g    205.5
dtype: float64 

均值： g    206.0
dtype: float64


+ alternative hypothesis : true location is not equal to 200
+ [scipy.stats.wilcoxon](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html)

In [14]:
# 與 200 的差異進行檢驗
result = stats.wilcoxon(a - 200, alternative='two-sided', correction = True, method='approx')
print(result)

WilcoxonResult(statistic=array([13.5]), pvalue=array([0.04977226]))


由上述結果可知中間值是 205.5，平均值是 206，p 值是 0.04977，因為 p 值小於顯著水準，所以拒絕虛無假設，中位數不等於 200 公克

## 範例2
```
郵局的辦事小姐處理隨機選取的 16 位顧客的郵寄業務
所花的時間如下：0.8 0.7 0.9 3.8 2.6 1.2 2.9 0.5 0.8 2.7 0.6 6.2 2.8 1.3 1.5 1.7，
檢定處理每件業務所花時間的中位數是否大於 1 分鐘？
```

In [15]:
b = pd.read_csv(r"C:\Users\user\Desktop\R 改 python\11_假說檢定\11a_one-sample-test-data\one-sample-test\b.csv")
b

,time
0,0.8
1,0.7
2,0.9
3,3.8
4,2.6
5,1.2
6,2.9
7,0.5
8,0.8
9,2.7


In [16]:
median = b.median(axis=0)
mean = b.mean(axis=0)
print("中位數：", median, '\n')
print("均值：", mean)

中位數： time    1.4
dtype: float64 

均值： time    1.9375
dtype: float64


+ alternative hypothesis : true location is less than 1

In [17]:
# 與 1 的差異進行檢驗
result = stats.wilcoxon(b - 1, alternative='less', correction = True, method='approx')
print(result)

WilcoxonResult(statistic=array([108.]), pvalue=array([0.98197257]))


由上述結果可知，中間值是 1.4，平均值是 1.9375，p 值是 0.982，因為 p 值大於顯著水準，所以不拒絕虛無假設，中位數時間大於 1 分鐘

## 範例3
```
某位品檢員想知道產品之中位數是否小於 82 公克？
他隨機抽取了生產線上 14 件產品檢驗，
得到下列數據：83.5 81.3 78.2 82 85.4 88.3 76.2 79.8 83.6 77.3 86.8 80.5 81.8 79.6。
```

In [18]:
c = pd.read_csv(r"C:\Users\user\Desktop\R 改 python\11_假說檢定\11a_one-sample-test-data\one-sample-test\c.csv")
c

,g
0,83.5
1,81.3
2,78.2
3,82.0
4,85.4
5,88.3
6,76.2
7,79.8
8,83.6
9,77.3


In [19]:
median = c.median(axis=0)
mean = c.mean(axis=0)
print("中位數：", median, '\n')
print("均值：", mean)

中位數： g    81.55
dtype: float64 

均值： g    81.735714
dtype: float64


+ alternative hypothesis : true location is greater than 82

In [20]:
# 與 82 的差異進行檢驗
result = stats.wilcoxon(c - 82, alternative='greater', correction = True, method='approx')
print(result)

WilcoxonResult(statistic=array([40.5]), pvalue=array([0.64969179]))


由上述結果可知，中間值是 81.55，平均值是 81.73571，p 值是 0.6497，因為 p 值大於顯著水準，所以不拒絕虛無假設，中位數小於 82 公克

### 備註1
+ [無母數統計—威爾卡森符號檢定](https://highscope.ch.ntu.edu.tw/wordpress/?p=73978)

### 備註2
+ [無母數統計—符號檢定](http://highscope.ch.ntu.edu.tw/wordpress/?p=73977)

---